**Mount Google Drive**

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Installing Flask-ngrok**

In [2]:
!pip install flask-ngrok

**Import Library**

In [3]:
import cv2
import numpy as np
import base64
from flask import Flask, request, render_template
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from flask_ngrok import run_with_ngrok
from google.colab.output import eval_js
import os

**Create Link Flask**

In [4]:
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://nw3gop3mq2a-496ff2e9c6d22116-5000-colab.googleusercontent.com/


**Running System**

In [13]:
app = Flask(__name__, template_folder='/content/gdrive/MyDrive/UM/templates')
run_with_ngrok(app)

# Muat model yang sudah dilatih
unet_model = load_model('/content/gdrive/MyDrive/UM/unet_model.h5')  # Model Segmentasi
cnn_model = load_model('/content/gdrive/MyDrive/UM/cnn_classification_model.h5')  # Model Klasifikasi
#resnet50_cnn = load_model('/content/gdrive/MyDrive/UM/Copy of resnet50_model.h5') # Model Klasifikasi
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('/content/gdrive/MyDrive/UM/classes.npy', allow_pickle=True)

def segment_image_with_unet(image):
    image_resized = cv2.resize(image, (256, 256))
    image_normalized = image_resized / 255.0
    segmented = unet_model.predict(
        image_normalized.reshape(1, 256, 256, 1))[0].reshape(256, 256)
    return segmented


def predict_class(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    segmented_image = segment_image_with_unet(image)

    # Untuk klasifikasi dengan CNN:
    img_array = np.expand_dims(segmented_image, axis=-1)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    predictions = cnn_model.predict(img_array)

    # Untuk klasifikasi dengan ResNet50 (jika digunakan):
    # img_array = np.stack((segmented_image,)*3, axis=-1) # Ubah ke RGB
    # img_array = np.expand_dims(img_array, axis=0)
    # predictions = resnet50_cnn.predict(img_array)

    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class = label_encoder.inverse_transform(
        [predicted_class_index])[0]

    # Encode gambar asli dan segmentasi ke base64
    _, buffer_original = cv2.imencode('.jpg', image)
    original_image_base64 = base64.b64encode(buffer_original).decode('utf-8')

    segmented_image_display = (segmented_image * 255).astype(np.uint8)
    _, buffer_segmented = cv2.imencode('.jpg', segmented_image_display)
    segmented_image_base64 = base64.b64encode(
        buffer_segmented).decode('utf-8')

    return predicted_class, original_image_base64, segmented_image_base64


@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        if 'file' not in request.files:
            return "No file part"
        file = request.files['file']
        if file.filename == '':
            return "No selected file"
        if file:
            file.save('uploaded_image.jpg')
            predicted_class, original_image_base64, segmented_image_base64 = predict_class(
                'uploaded_image.jpg')
            return render_template('result.html', prediction=predicted_class,
                                   original_image=original_image_base64,
                                   segmented_image=segmented_image_base64)
    return render_template('index.html')


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-44:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 199, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 789, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step


INFO:werkzeug:127.0.0.1 - - [03/Dec/2024 15:11:56] "POST /?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Dec/2024 15:11:57] "GET /favicon.ico?authuser=0 HTTP/1.1" 404 -
